Installing All required Libaries

In [1]:
!pip install -U langchain umap-learn langchain_community tiktoken langchainhub chromadb
!pip install sentence-transformers
!pip install html2text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

Importing All Required Libaries

In [2]:
from langchain_community.document_transformers import Html2TextTransformer
from langchain_community.vectorstores import Chroma
import nest_asyncio
from langchain_community.embeddings import HuggingFaceEmbeddings
import tiktoken
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
import torch
from huggingface_hub import login
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import RetrievalQA
import os
from langchain_community.llms import HuggingFaceHub


Loading and reading the data for Metlife

In [3]:

# Metlife
url = "https://www.metlife.com/stories/accident-health/what-is-accident-insurance/"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docsinc = loader.load()
# Metlife docs
url = "https://www.metlife.com/insurance/accident-health/accident-insurance/"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# Metlife
url = "https://www.metlife.com/insurance/accident-health/hospital-indemnity-insurance/"
loader = RecursiveUrlLoader(
    url=url, max_depth=1, extractor=lambda x: Soup(x, "html.parser").text
)
docs_pydantic = loader.load()

# Metlife
url = "https://www.metlife.com/insurance/accident-health/critical-illness-insurance/"
loader = RecursiveUrlLoader(
    url=url, max_depth=1, extractor=lambda x: Soup(x, "html.parser").text
)
docs_sq = loader.load()

# Doc texts
# docs.extend([*docs_pydantic, *docs_sq, *docsinc])
docs.extend(
    [
        *docs_pydantic,
        *docs_sq,
    ]
)


# Converts HTML to plain text
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

In [4]:

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens



Chunk text

In [5]:
# Chunk text
# Doc texts concat

d_sorted = sorted(docs_transformed, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)
print(
    "Num tokens in all context: %s"
    % num_tokens_from_string(concatenated_content, "cl100k_base")
)

# Doc texts split
chunk_size_tok = 2000
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=chunk_size_tok, chunk_overlap=0
)
texts_split = text_splitter.split_text(concatenated_content)


Num tokens in all context: 9809


Loading mpnet

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
# Load chunked documents into the Vectorstore index
vectorstore = Chroma.from_texts(texts=texts_split, embedding=embeddings)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Defing Retriever

In [7]:
# retriever = vectorstore.as_retriever()
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})


loading Mistral-7B-Instruct-v0.2  

In [8]:
token = "hf_xxxxxxxxxxxxxxxxxxxxx"


login(token)
print(torch.__version__)
# print(torch.zeros(1).cuda())
print(torch.cuda.is_available())
device = torch.device("cpu")

os.environ["HUGGINGFACEHUB_API_TOKEN"] = token

hf = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    model_kwargs={"temperature": 0.1, "max_length": 300},
)


llm = hf


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful
2.2.1+cu121
False


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


Preparing Prompt Template

In [10]:
prompt_template = """
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

retrievalQA = RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)
query = "What does the accident insurance cover?"

# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result["result"])



Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

Accident Insurance | MetLife Skip Navigation Navigation SOLUTIONS SUPPORT ABOUT US RESOURCES LOG IN LOG IN SEARCH SEARCH close LOG IN LOG IN SEARCH SEARCH Close SOLUTIONS Insurance Business and Brokers MetLife Investment Management Retirement & Income Solutions Insurance Enhance Your Health Coverage Dental Vision Pet Insurance Protect Your Finances Accident Hospital Indemnity Critical Illness Cancer Disability and Absence Safeguard Your Valuables Auto and Home Insurance Legal Plans Identity & Fraud Protection Secure Your Family's Future Term Life Group Variable Universal Life Group Universal Life Insurance Accident Death & Dismemberment Reach Your Financial Goals Health Savings and Spending Accounts Total Control Account (TCA) Employer-Sponsored Retirement Plans More Solutions Annuity Long-Term Care Business and Brokers Employee Benefits Group Auto & Home Group 

This code sets up a pipeline for using a RAG model to process input queries. It retrieves a pre-defined prompt, defines post-processing functions for formatting documents, sets up the sequence of processing steps, and invokes the pipeline to process a given query.

In [11]:
# Prompt
prompt = hub.pull("rlm/rag-prompt")


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
#
# # Question
rag_chain.invoke(query)


'Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don\'t know the answer, just say that you don\'t know. Use three sentences maximum and keep the answer concise.\nQuestion: What does the accident insurance cover? \nContext: Accident Insurance | MetLife Skip Navigation Navigation SOLUTIONS SUPPORT ABOUT US RESOURCES LOG IN LOG IN SEARCH SEARCH close LOG IN LOG IN SEARCH SEARCH Close SOLUTIONS Insurance Business and Brokers MetLife Investment Management Retirement & Income Solutions Insurance Enhance Your Health Coverage Dental Vision Pet Insurance Protect Your Finances Accident Hospital Indemnity Critical Illness Cancer Disability and Absence Safeguard Your Valuables Auto and Home Insurance Legal Plans Identity & Fraud Protection Secure Your Family\'s Future Term Life Group Variable Universal Life Group Universal Life Insurance Accident Death & Dismemberment Reach Your Financial Goals Health Savings an